In [ ]:

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

# from dynamicslib.common import f_
from dynamicslib.consts import muEM
from dynamicslib.continuation import f_df_CR3_single, find_bif, arclen_cont, arclen_to_fail
from dynamicslib.common import prop_ic, get_JC_tf, eom
from dynamicslib.integrator import dop853
from tqdm.auto import tqdm
from dynamicslib.plotting import plotly_display

from dynamicslib.targeter import dc_npc


# muEM = 1e-2


In [ ]:
# X: [x, tf]
# f: [y, dx]
# lam: vy
def xtf2X_npc(x0, tf):
    return np.array([x0[0], tf / 2])


def X2xtf_npc(X, param):
    return np.array([X[0], 0, 0, 0, param, 0]), X[-1] * 2


def stmeom2DF_npc(eomf, stm):
    dF = np.array([[stm[1, 0], eomf[1]], [stm[-3, 0], eomf[-3]]])
    return dF


def f_func_npc(x0, tf, xf):
    return np.array([xf[1], xf[-3]])


func_npc = lambda param: lambda X: f_df_CR3_single(
    X, lambda X: X2xtf_npc(X, param), stmeom2DF_npc, f_func_npc, False, muEM, 1e-10
)


def xtf2X(x0, tf):
    return np.array([x0[0], x0[-2], tf / 2])


def X2xtf(X):
    return np.array([X[0], 0, 0, 0, X[1], 0]), X[-1] * 2


def stmeom2DF(eomf, stm):
    dF = np.array(
        [[stm[1, 0], stm[1, -2], eomf[1]], [stm[-3, 0], stm[-3, -2], eomf[-3]]]
    )
    return dF


def f_func(x0, tf, xf):
    return np.array([xf[1], xf[-3]])


func = lambda X: f_df_CR3_single(X, X2xtf, stmeom2DF, f_func, False, muEM, 1e-10)


def get_IC_kep(p, q, e: float = 0.2):
    period = 2 * np.pi * q

    a = (q / p) ** (2 / 3)

    r0 = a * (1 + e)
    v0_inert = np.sqrt(2 / r0 - 1 / a)

    vrot = r0
    v0 = v0_inert - vrot
    return r0, v0, period


# get the linear IC
def get_IC(
    p,
    q,
    e: float = 0.2,
    tol: float = 1e-12,
    mu: float = muEM,
    start_right: bool = True,
    fudge: float = 1,
    debug: bool = False,
):
    # p orbits of the SC for every q orbits of the primary

    r0, v0, period = get_IC_kep(p, q, e)

    # x0_guess = np.array([r0, 0, 0, 0, v0, 0])
    print("STARTING")
    print(period)
    print(r0)
    x0 = r0 - mu if start_right else -mu - r0
    v0 = v0 if start_right else -v0
    X, _, _ = dc_npc([x0, period / 2], func_npc(v0), fudge=fudge, debug=debug, tol=tol)
    X0 = np.array([X[0], v0, X[-1]])
    _, dF, stm = func(X0)
    svd = np.linalg.svd(dF)
    tangent = svd.Vh[-1]
    eigs = np.linalg.eigvals(stm)
    return X0, tangent, eigs
    # return [r0, v0, period / 2]

In [ ]:
import matplotlib.pyplot as plt

# r0, v0, period = get_IC_kep(2, 3, 0.3, muEM)
X0, tangent, e0 = get_IC(3, 4, 0.7, fudge=0.5, debug=False, start_right=False)
xyz = prop_ic(X0, X2xtf, mu=muEM)
plt.plot(xyz[0], xyz[1])
plt.scatter(xyz[0][0], xyz[1][0], c="b")
plt.scatter(xyz[0][-1], xyz[1][-1], c="r")
plt.axis("equal")

In [ ]:
# X0, tangent, e0 = get_IC(2, 5, 0.6, True)

In [ ]:
Xs, es = arclen_cont(X0, func, tangent, 5e-3, .5, 1e-11, exact_tangent=True,max_iter=30)

In [ ]:
Xsp = [*Xs]
esp = [*es]

param_names = [
    "Index",
    "Initial x",
    "Initial vy",
    "Period",
    "Jacobi Constant",
    "Stability Index",
    "Eig1",
    "Eig2",
    "Eig3",
    "Eig4",
    "Eig5",
    "Eig6",
]

xyzs = []
data = []

iterator = list(enumerate(zip(Xsp, esp)))
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    xyzs.append(prop_ic(Xcp, X2xtf, mu=muEM))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc, period = get_JC_tf(Xcp, X2xtf, mu=muEM)
    data.append([i, *Xcp[:-1], period, jc, stabind, *evals])

In [ ]:
df = pd.DataFrame(data, columns=param_names).set_index("Index")
df.to_csv("Resonant 3-4.csv")

a = None
b = None
c = 3
plotly_display(xyzs[a:b:c], df[a:b:c], mu=muEM)